In [ ]:
import h5py
import numpy as np
import open3d as o3d
import os

data_names = ['positions', 'shape_quats', 'scene_params']

def load_data(data_names, path):
    hf = h5py.File(path, 'r')
    data = []
    for i in range(len(data_names)):
        d = np.array(hf.get(data_names[i]))
        data.append(d)
    hf.close()
    return data


def store_data(data_names, data, path):
    hf = h5py.File(path, 'w')
    for i in range(len(data_names)):
        hf.create_dataset(data_names[i], data=data[i])
    hf.close()


def flip_inward_normals(pcd, center, threshold=0.7):
    # Flip normal if normal points inwards by changing vertex order
    # https://math.stackexchange.com/questions/3114932/determine-direction-of-normal-vector-of-convex-polyhedron-in-3d
    
    # Get vertices and triangles from the mesh
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    # For each triangle in the mesh
    flipped_count = 0
    for i, n in enumerate(normals):
        # Compute vector from 1st vertex of triangle to center
        norm_ref = points[i] - center
        # Compare normal to the vector
        if np.dot(norm_ref, n) < 0:
            # Change vertex order to flip normal direction
            flipped_count += 1 
            if flipped_count > threshold * normals.shape[0]:
                normals = np.negative(normals)
                break

    pcd.normals = o3d.utility.Vector3dVector(normals)
    # o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    return pcd


def reconstruct_mesh_from_pcd(cube, algo="poisson"):
    if algo == "alpha_shape":
        tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(cube)
        alpha = 0.05
        # print(f"alpha={alpha:.3f}")
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
            cube, alpha, tetra_mesh, pt_map)
        mesh.compute_vertex_normals()
    else: 
        # cube = o3d.io.read_point_cloud(f"cube/gripper_ngrip_{frame}.ply")
        cube.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals
        cube.estimate_normals()
        cube.orient_normals_consistent_tangent_plane(100)
        center = cube.get_center()
        cube = flip_inward_normals(cube, center)

        if algo == "ball_pivot":
            radii = [0.005, 0.01, 0.02, 0.04]
            mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                cube, o3d.utility.DoubleVector(radii))
        elif algo == "poisson":
            with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
                mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(cube, depth=8)

    # o3d.visualization.draw_geometries([cube, mesh], mesh_show_back_face=True)
    return mesh

In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt

%matplotlib inline

def visualize_points(ax, all_points, n_particles):
    points = ax.scatter(all_points[:n_particles, 0], all_points[:n_particles, 2], all_points[:n_particles, 1], c='b', s=10)
    shapes = ax.scatter(all_points[n_particles:, 0], all_points[n_particles:, 2], all_points[n_particles:, 1], c='r', s=10)
    
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:, 1] - extents[:, 0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = 0.25  # maxsize / 2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

    ax.invert_yaxis()

    return points, shapes


def plt_render(particles_set, n_particle, render_path):
    # particles_set[0] = np.concatenate((particles_set[0][:, :n_particle], particles_set[1][:, n_particle:]), axis=1)
    n_frames = particles_set[0].shape[0]
    rows = 2
    cols = 3

    fig, big_axes = plt.subplots(rows, 1, figsize=(9, 6))
    row_titles = ['GT', 'Sample']
    views = [(90, 90), (0, 90), (45, 135)]
    plot_info_all = {}
    for i in range(rows):
        big_axes[i].set_title(row_titles[i], fontweight='semibold')
        big_axes[i].axis('off')

        plot_info = []
        for j in range(cols):
            ax = fig.add_subplot(rows, cols, i * cols + j + 1, projection='3d')
            ax.view_init(*views[j])
            points, shapes = visualize_points(ax, particles_set[i][0], n_particle)
            plot_info.append((points, shapes))

        plot_info_all[row_titles[i]] = plot_info

    plt.tight_layout()
    # plt.show()

    def update(step):
        outputs = []
        for i in range(rows):
            states = particles_set[i]
            for j in range(cols):
                points, shapes = plot_info_all[row_titles[i]][j]
                points._offsets3d = (states[step, :n_particle, 0], states[step, :n_particle, 2], states[step, :n_particle, 1])
                shapes._offsets3d = (states[step, n_particle:, 0], states[step, n_particle:, 2], states[step, n_particle:, 1])
                outputs.append(points)
                outputs.append(shapes)
        return outputs

    anim = animation.FuncAnimation(fig, update, frames=np.arange(0, n_frames), blit=False)
    
    # plt.show()
    anim.save(render_path, writer=animation.PillowWriter(fps=20))

In [ ]:
task_name = 'ngrip_fixed_v3'
data_path = f'/scr/hxu/projects/deformable/VGPL-Dynamics-Prior/data/data_{task_name}'
new_data_path = f'/scr/hxu/projects/deformable/VGPL-Dynamics-Prior/data/data_{task_name}_updated'

n_points = 300
n_frames = 119

vid_info = {'train': 90, 'valid': 10}

for name, n_vid in vid_info.items():
    for i in range(n_vid):
        rollout_path = os.path.join(new_data_path, name, f'{i:03d}')
        os.system(f'mkdir -p {rollout_path}')
        all_positions = []
        all_gt_positions = []
        for k in range(0, n_frames):
            shape_data = load_data(data_names, os.path.join(data_path, name, f'{i:03d}', 'shape_'+str(k)+'.h5'))
            shape_gt_data = load_data(data_names, os.path.join(data_path, name, f'{i:03d}', 'shape_gt_'+str(k)+'.h5'))
            points = shape_data[0][:n_points]

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(points)
            mesh = reconstruct_mesh_from_pcd(pcd, algo='alpha_shape')

            pcd_new = o3d.geometry.TriangleMesh.sample_points_poisson_disk(mesh, n_points)

            position = np.concatenate((np.asarray(pcd_new.points), shape_data[0][n_points:]))
            gt_position = shape_gt_data[0]
            
            
            all_positions.append(position)
            all_gt_positions.append(shape_data[0])

            # pcd_new.colors = np.tile([0.0, 0.0, 0.0], (n_points_new, 1))
            # o3d.visualization.draw_geometries([pcd_new], mesh_show_back_face=True)
            
            shape_data[0] = position
            
            store_data(data_names, shape_data, os.path.join(rollout_path, 'shape_'+str(k)+'.h5'))
            store_data(data_names, shape_gt_data, os.path.join(rollout_path, 'shape_gt_'+str(k)+'.h5'))

        plt_render([np.array(all_gt_positions), np.array(all_positions)], n_points, os.path.join(rollout_path, 'plt.gif'))